# NIN块

NIN块是NIN中的基础块，他由一个卷积层加两个充当全连接层的1*1卷积层串联而成，其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

In [1]:
import torch
from torch import nn,optim

def ninBlock(inChannels,outChannels,kernelSize,stride,padding):

    block = nn.Sequential(
        nn.Conv2d(inChannels,outChannels,kernelSize,stride,padding),
        nn.ReLU(),
        nn.Conv2d(outChannels,outChannels,1),
        nn.ReLU(),
        nn.Conv2d(outChannels,outChannels,1),
        nn.ReLU()
    )

    return block

# NIN模型

NIN使用卷积窗口形状分别为11*11、5*5、3*3的卷积层，相应的输出通道数也与AlexNet中的一直。每个后面接一个歩幅为2、窗口大小为3*3的最大池化层。  
NIN去掉了AlexNet最后的3个全连接层，使用输出通道等于标签类别数的NIN块，然后使用全局平均池化对每个通道中的元素求平均并直接用于分类。

In [2]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return torch.nn.functional.adaptive_avg_pool2d(x, (1,1))

net=nn.Sequential(
        ninBlock(1,96,11,4,0),
        nn.MaxPool2d(3,2),
        ninBlock(96,256,5,1,2),
        nn.MaxPool2d(3,2),
        ninBlock(256,384,3,1,1),
        nn.MaxPool2d(3,2),
        nn.Dropout(0.5),
        ninBlock(384,10,3,1,1),
        GlobalAvgPool2d(),
        nn.Flatten()
        )
x=torch.rand(1,1,224,224)
for name,blk in net.named_children():
    x=blk(x)
    print(name,x.shape)
        

0 torch.Size([1, 96, 54, 54])
1 torch.Size([1, 96, 26, 26])
2 torch.Size([1, 256, 26, 26])
3 torch.Size([1, 256, 12, 12])
4 torch.Size([1, 384, 12, 12])
5 torch.Size([1, 384, 5, 5])
6 torch.Size([1, 384, 5, 5])
7 torch.Size([1, 10, 5, 5])
8 torch.Size([1, 10, 1, 1])
9 torch.Size([1, 10])


# 读取数据集

In [3]:
import torchvision

transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(size=224),
    torchvision.transforms.ToTensor()]
)

#获取数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transform)
#读取数据集
batchSize=128
trainIter=torch.utils.data.DataLoader(mnist_train,batch_size=batchSize,shuffle=True,num_workers=8)
testIter=torch.utils.data.DataLoader(mnist_test,batch_size=batchSize,shuffle=True,num_workers=8)

# 评价


In [4]:
def evaluate_accuracy(data_iter, net):
    device=torch.device('cuda')
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        net.eval() # 评估模式
        for X, y in data_iter:
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().item()
            n += y.shape[0]
        net.train() # 改回训练模式 
    return acc_sum / n

# 训练

In [5]:
lr,epochsNum = 0.002,5
net=net.cuda()
loss=torch.nn.CrossEntropyLoss()
optimzer=torch.optim.Adam(net.parameters(),lr)

for epoch in range(epochsNum):
    train_l_sum=n=train_acc_sum=0
    for x,y in trainIter:
        x=x.cuda()
        y=y.cuda()
        yP=net(x)
        l=loss(yP,y)
        l.backward() 
        optimzer.step()
        optimzer.zero_grad()
        train_l_sum += l.item()
        train_acc_sum += (yP.argmax(dim=1) == y).sum().item()
        n +=y.shape[0]
    print(epoch,train_l_sum/n,train_acc_sum/n,evaluate_accuracy(testIter,net))

0 0.010261349797248841 0.5344333333333333 0.7542
1 0.0045598106841246286 0.7843333333333333 0.8074
2 0.0037446690648794173 0.8207666666666666 0.8339
3 0.0034412108908096948 0.8354333333333334 0.8483
4 0.0032475295327603817 0.8451333333333333 0.8451
